# FastAI Movie Recommender Step-by-Step
> An app where you can type in your favorite movie and it will suggest 30 movies similar to i t using a collaborative filtering model trained on over 9million movie reviews.

In [ ]:
#| default_exp movie_recommender

### Makes sure all your libraries and packages are installed and loaded.

[Fastai](https://youtu.be/p4ZZq0736Po?t=3724) is the library we use to train, load and run our model.  NB: we already trained our model so we'll only be loading it here.

In [ ]:
# !pip install -Uqq fastai 

[Gradio](https://gradio.app/getting_started/) allows you to easily publish your model onto the web for peope to use.

In [ ]:
# !pip install -Uqq gradio

[nbdev](https://nbdev.fast.ai/tutorials/tutorial.html) is what we're using to make this documentation easier and more presentable.

In [ ]:
# !pip install nbdev

In [ ]:
#| hide
from nbdev.showdoc import *

These are the libraries we need

In [ ]:
#| export
# For modeling
from fastai.tabular.all import *
from fastai.collab import *
#for publishing the model
import gradio as gr

In [ ]:
# For modeling
from fastai.tabular.all import *
from fastai.collab import *
#for publishing the model
import gradio as gr

### Load model and data loaders or movie title csv


The dataloaders we used to train the model from the 9 million reviews were 200MB so we use a csv instead. To convert from data loaders to pandas DataFrame use code hashtagged out just below

In [ ]:
# titles = pd.DataFrame(dls.classes['title']).reset_index() #create df from data loaders(dls)
# titles = titles.rename(columns={'index':'midx',0:'title'}) # rename columns

In [ ]:
#| export
#load the model and movie titles with indexes (from the previous data loaders)
path = Path('.')
data_folder = '../models_and_dls'

#check if modedels folder is up one level if not 'for readme' then check in current directory
if not os.path.isdir(data_folder):
    data_folder = './models_and_dls'
#load learner and titles csv    
learn = load_learner(path/data_folder/'movie_18mil_xu_mdl_669.pkl')
df_titles = pd.read_csv(path/data_folder/'movie_18mil_xu.csv')

### Functions for Model Inference

In [ ]:
type(df_titles)

pandas.core.frame.DataFrame

In [ ]:
#| export
def get_movie_recs(full_title:str, # String saved to `favorite_movie` variable
                   learn=learn ,# Trained Model
                   df_titles=df_titles #DataFrame with all movie titles from dls
                        ):
    "This function will use the `nn.CosineSimilarity` on the 50 latent factors for each movie to find the 30 movies most similar to your favorite movie."
    movie_factors = learn.model.i_weight.weight
    idx = int(df_titles[df_titles.title == full_title].midx)
    distances =  nn.CosineSimilarity(dim=1)(movie_factors,movie_factors[idx][None])
    idx = distances.argsort(descending=True)
    return [o for o in df_titles.title[idx.tolist()][:30]]

In [ ]:
show_doc(get_movie_recs)

---

[source](https://github.com/daveramseymusic/movie-rec/blob/main/movie_rec/movie_recommender.py#L26){target="_blank" style="float:right; font-size:smaller"}

### get_movie_recs

>      get_movie_recs (full_title:str, learn=<fastai.learner.Learner object at
>                      0x7f8ca03b7c50>, df_titles=      midx
>                      title 0        0                            #na# 1
>                      1              'burbs, The (1989) 2        2     (500)
>                      Days of Summer (2009) 3        3  *batteries not included
>                      (1987) 4        4   ...And Justice for All (1979) ...
>                      ...                             ... 4214  4214
>                      [REC] (2007) 4215  4215                 eXistenZ (1999)
>                      4216  4216                      xXx (2002) 4217  4217
>                      xXx: State of the Union (2005) 4218  4218
>                      ¡Three Amigos! (1986)  [4219 rows x 2 columns])

This function will use the `nn.CosineSimilarity` on the 50 latent factors for each movie to find the 30 movies most similar to your favorite movie.

In [ ]:
#| export
def search_movie_titles(favorite_movie:str, # String saved to `favorite_movie` variable
                        df_titles:pd.DataFrame): #DataFrame with all movie titles located in the column `title` from dls
    "This Function searches through df_titles['title'] column for titles that contian words saved in the `favorite_movie` variable after making all words lowercase and removing THE from the start of a title"
    movies_found = ''
    s = favorite_movie.lower()
    #remove THE from the title
    if s[:4] == 'the ':
        s = s[4:]

    lst = df_titles['title'].tolist() # turn it to a list
    index = []
    i=0
    length = len(lst)
    while i<length:
        if s in lst[i].lower():
            full_title = lst[i]
            movies_found+= str(full_title) +'\n'
#             print(f'Your Favorite Movies:  {full_title}')
        i+=1
    return full_title, movies_found

In [ ]:
#| export
def search_movies_n_recommend(favorite_movie:str, # The movie title typed into the `gr.Textbox()` that the user will see on the `gradio` app
                             learn, # Trained model
                              df_titles, # Movie titles df
                             ):
    "This function returns 30 recommendations using `get_movie_recs` after searching through all movie titles to find any titles that contain the words in the `favorite_movie` variable."
    full_title, movies_found = search_movie_titles(favorite_movie, df_titles=df_titles)    
    # write explaination in case multiple movies
    explainer = f'If there are multiple movies above: Please paste your favorite movie into the "favorite_movie" box.\n The box below is currently showing recommendations for the movie:  {full_title}'
    #print movies found and explainer
    output_str =  movies_found+'\n\n'+explainer
    
    #get recommendations from model
    recommendations = get_movie_recs(full_title=full_title, learn=learn, df_titles=df_titles) 
    
    #create list of all the recommendations to print
    print_lst = ''
    for o in recommendations:    print_lst =print_lst+str(o) + '\n'
    
    return output_str, f'Recommendations for {full_title}: \n\n {print_lst}'

In [ ]:
def search_movies_n_recommend(favorite_movie:str, # The movie title typed into the `gr.Textbox()` that the user will see on the `gradio` app
                             learn=learn, # Trained model
                              df_titles=df_titles, # Movie titles df
                             ):
    "This function returns 30 recommendations using `get_movie_recs` after searching through all movie titles to find any titles that contain the words in the `favorite_movie` variable."
    full_title, movies_found = search_movie_titles(favorite_movie, df_titles)    

    # write explaination in case multiple movies
    explainer = f'If there are multiple movies above: Please paste your favorite movie into the "favorite_movie" box.\n The box below is currently showing recommendations for the movie:  {full_title}'

    #print movies found and explainer
    output_str =  movies_found+'\n\n'+explainer
    
    #get recommendations from model
    recommendations = get_movie_recs(full_title=full_title, learn=learn, df_titles=df_titles) 

    #create list of all the recommendations to print
    print_lst = ''
    for o in recommendations:    print_lst =print_lst+str(o) + '\n'
    
    return output_str, f'Recommendations for {full_title}: \n\n {print_lst}'

### Gradio For Publishing the Model 
>Now we use `gr.Interface()` to create a block that contains our `search_movies_n_recommend` fuction as well as variables that dictate what will be shown once we publish it on `gradio`

In [ ]:
intf = gr.Interface(fn=search_movies_n_recommend,
                    inputs=gr.Textbox(lines=1,placeholder="Put Your Favorite Movie Here To See Similar Movie Recommendations..."),
                    outputs=["text","text"],
                    examples=['Child\'s Play (1988)','Rushmore','brothers Bloom',
                              'call me by your name', 'Jumanji (1995)','The Three Amigos'],
                    title="Dave's Fast.AI Movie Recommender",
                    article='This recommender uses a collaborative filtering model fashioned from the [Fast.AI](https://github.com/fastai/fastbook/blob/master/08_collab.ipynb) library and trained on the 9 million movie reviews from the [Movie Lense Dataset](https://grouplens.org/datasets/movielens/25m/).  It trains Latent Factors to give movies and users descriptive weights that help predict thier reivews based on these factors.  The model was trained completely WITHOUT titles, descriptions, or categories and amazingly these things can be added to imporove this model.  Enjoy!')

# Actually Try the Model HERE
>Now we launch it so it can be used literally here in the `jupyter notebook`

In [ ]:
intf.launch(inline=True, #when True publishes the page in notebook
        share=False) #when True automatically publishes the app to the gradio website for 72 hours. 

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App>, 'http://127.0.0.1:7861/', None)

## Example Without Gradio: 
>Here is an example of the model being used straight from the `get_movie_recs` function, recommending movies similar to one of my favorites.

In [ ]:
favorite_movie = 'Beasts of the Southern Wild (2012)'
get_movie_recs(full_title=favorite_movie, learn=learn, df_titles=df_titles)

['Beasts of the Southern Wild (2012)',
 'No Country for Old Men (2007)',
 'Wrestler, The (2008)',
 'Leaving Las Vegas (1995)',
 'City of God (Cidade de Deus) (2002)',
 'Talk to Her (Hable con Ella) (2002)',
 'Antichrist (2009)',
 'Nocturnal Animals',
 'Requiem for a Dream (2000)',
 'Magnolia (1999)',
 'In the Company of Men (1997)',
 'Old Boy (2003)',
 'Enemy (2013)',
 '12 Angry Men (1957)',
 'Samouraï, Le (Godson, The) (1967)',
 'Bringing Out the Dead (1999)',
 'Eyes Wide Shut (1999)',
 'Mulholland Drive (2001)',
 'Punch-Drunk Love (2002)',
 'Shallow Grave (1994)',
 'Birdman: Or (The Unexpected Virtue of Ignorance) (2014)',
 'Taxi Driver (1976)',
 'Black Swan (2010)',
 'American Beauty (1999)',
 'Broken Flowers (2005)',
 "Winter's Bone (2010)",
 'Little Miss Sunshine (2006)',
 'Hunt, The (Jagten) (2012)',
 'Control (2007)',
 'The Machinist (2004)']

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()